In [ ]:
!pip install txgraffiti

In [ ]:
# data load cell
import re
import requests
import pandas as pd

# Raw GitHub URLs (public)
BASE = "https://raw.githubusercontent.com/TomasSilva/MLcCY7/main/Data"
URL_WP4S = f"{BASE}/WP4s.txt"
URL_HODGES = f"{BASE}/WP4_Hodges.txt"

def fetch_text(url: str, timeout: int = 60) -> str:
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    return r.text

def parse_int_rows(text: str, expected_len: int) -> list[list[int]]:
    rows = []
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue
        ints = list(map(int, re.findall(r"-?\d+", line)))
        if len(ints) != expected_len:
            raise ValueError(f"Unexpected row width {len(ints)} (expected {expected_len}) in line: {line[:120]}")
        rows.append(ints)
    return rows

# Download
wp4s_txt = fetch_text(URL_WP4S)
hodges_txt = fetch_text(URL_HODGES)

# Parse
W = parse_int_rows(wp4s_txt, expected_len=5)      # w1..w5
H = parse_int_rows(hodges_txt, expected_len=2)    # h11, h21

if len(W) != len(H):
    raise ValueError(f"Row-count mismatch: weights={len(W)} vs hodges={len(H)}")

# Build dataframe
df = pd.DataFrame(W, columns=["w1","w2","w3","w4","w5"])
df[["h11","h21"]] = pd.DataFrame(H, columns=["h11","h21"])

# Common derived invariant for CY3s
df["euler_chi"] = 2 * (df["h11"] - df["h21"])

df


ValueError: Unexpected row width 37775 (expected 5) in line: [[1, 1, 1, 1, 1], [1, 1, 1, 1, 2], [1, 1, 1, 1, 3], [1, 1, 1, 2, 2], [1, 1, 1, 1, 4], [1, 1, 1, 2, 3], [1, 1, 2, 2, 2], 

In [ ]:
import json
import ast
import re
import requests
import pandas as pd

BASE = "https://raw.githubusercontent.com/TomasSilva/MLcCY7/main/Data"
URL_WP4S   = f"{BASE}/WP4s.txt"
URL_HODGES = f"{BASE}/WP4_Hodges.txt"

def fetch_text(url: str, timeout: int = 60) -> str:
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    return r.text

def parse_table(text: str, expected_len: int):
    s = text.strip()

    # Case 1: whole-file literal like [[...],[...],...]
    if s.startswith("["):
        try:
            data = json.loads(s)          # works if it's valid JSON
        except Exception:
            data = ast.literal_eval(s)    # works if it's Python literal
        if not (isinstance(data, list) and all(isinstance(row, (list, tuple)) for row in data)):
            raise ValueError("Parsed literal is not a list of rows.")
        for i, row in enumerate(data[:10]):
            if len(row) != expected_len:
                raise ValueError(f"Row {i} has length {len(row)} (expected {expected_len}).")
        return [list(map(int, row)) for row in data]

    # Case 2: one row per line (space/comma separated)
    rows = []
    for line in s.splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        ints = list(map(int, re.findall(r"-?\d+", line)))
        if len(ints) != expected_len:
            raise ValueError(f"Unexpected row width {len(ints)} (expected {expected_len}) in line: {line[:120]}")
        rows.append(ints)
    return rows

# Download
wp4s_txt   = fetch_text(URL_WP4S)
hodges_txt = fetch_text(URL_HODGES)

# Parse
W = parse_table(wp4s_txt, expected_len=5)   # weights w1..w5
H = parse_table(hodges_txt, expected_len=2) # (h11, h21)

if len(W) != len(H):
    raise ValueError(f"Row-count mismatch: weights={len(W)} vs hodges={len(H)}")

# Build df
df = pd.DataFrame(W, columns=["w1","w2","w3","w4","w5"])
df[["h11","h21"]] = pd.DataFrame(H, columns=["h11","h21"])
df["euler_chi"] = 2 * (df["h11"] - df["h21"])

df


,w1,w2,w3,w4,w5,h11,h21,euler_chi
0,1,1,1,1,1,1,101,-200
1,1,1,1,1,2,1,103,-204
2,1,1,1,1,3,2,122,-240
3,1,1,1,2,2,2,95,-186
4,1,1,1,1,4,1,149,-296
...,...,...,...,...,...,...,...,...
7550,31,42,396,938,1407,462,12,900
7551,28,41,414,966,1449,416,14,804
7552,36,41,421,996,1494,491,11,960
7553,36,41,462,1078,1617,462,12,900


In [ ]:
import numpy as np
import math
from functools import reduce

def igcd(a, b):
    return math.gcd(int(a), int(b))

def ilcm(a, b):
    a, b = int(a), int(b)
    return abs(a*b) // math.gcd(a, b) if a and b else 0

def gcd_list(xs):
    return reduce(igcd, xs)

def lcm_list(xs):
    return reduce(ilcm, xs, 1)

wcols = ["w1","w2","w3","w4","w5"]
W = df[wcols].to_numpy(dtype=int)

# ---------- Numeric columns (4) ----------
df["w_sum"]    = W.sum(axis=1)                                 # sum of weights
df["w_prod"]   = np.prod(W, axis=1)                            # product of weights
df["w_gcd"]    = np.array([gcd_list(row) for row in W], int)   # gcd(w_i)
df["w_lcm"]    = np.array([lcm_list(row) for row in W], int)   # lcm(w_i)

# (you already have) df["euler_chi"] = 2*(h11-h21)

# ---------- Boolean columns (4) ----------
# 1) Any repeated weight? (symmetry / degeneracy)
df["has_repeat_weight"] = np.array([len(set(row)) < 5 for row in W])

# 2) Pairwise coprime? (strong arithmetic condition)
df["pairwise_coprime"] = np.array([
    all(math.gcd(row[i], row[j]) == 1 for i in range(5) for j in range(i+1, 5))
    for row in W
])

# 3) "Degree divisibility" heuristic: d = sum(w_i) divisible by each w_i
# (common in weighted hypersurface bookkeeping; good for patterns)
df["sum_divisible_by_all_weights"] = np.array([
    (row.sum() % row == 0).all() for row in W
])

# 4) Parity pattern: exactly one even weight (often yields crisp conjectures)
df["exactly_one_even_weight"] = np.array([
    (np.sum(row % 2 == 0) == 1) for row in W
])

df.head()


,w1,w2,w3,w4,w5,h11,h21,euler_chi,w_sum,w_prod,w_gcd,w_lcm,has_repeat_weight,pairwise_coprime,sum_divisible_by_all_weights,exactly_one_even_weight
0,1,1,1,1,1,1,101,-200,5,1,1,1,True,True,True,False
1,1,1,1,1,2,1,103,-204,6,2,1,2,True,True,True,True
2,1,1,1,1,3,2,122,-240,7,3,1,3,True,True,False,False
3,1,1,1,2,2,2,95,-186,7,4,1,2,True,False,False,False
4,1,1,1,1,4,1,149,-296,8,4,1,4,True,True,True,True


In [ ]:

import pandas as pd

# at the top of graffiti4.py
from txgraffiti.graffiti3.heuristics.morgan import morgan_filter
from txgraffiti.graffiti3.heuristics.dalmatian import dalmatian_filter
from txgraffiti.graffiti3.graffiti3 import Graffiti3, Stage


g3 = Graffiti3(
    df,
    max_boolean_arity=2,
    morgan_filter=morgan_filter,
    dalmatian_filter=dalmatian_filter,
    sophie_cfg=dict(
        eq_tol=1e-4,
        min_target_support=5,
        min_h_support=3,
        max_violations=0,
        min_new_coverage=1,
    ),
)


# The stages add additional complexity. Uncomment them slowly and observe
STAGES = [
    Stage.CONSTANT,
    Stage.RATIO,
    Stage.LP1,
    # Stage.LP2,
    # Stage.LP3,
    # Stage.LP4,
    # Stage.POLY_SINGLE,
    # Stage.MIXED,
    # Stage.SQRT,
    # Stage.LOG,
    # Stage.SQRT_LOG,
    Stage.GEOM_MEAN,
    # Stage.LOG_SUM,
    # Stage.SQRT_PAIR,
    # Stage.SQRT_SUM,
    # Stage.EXP_EXPONENT,

]

# Target invariants to conjecture on. Anything you include here is a column in the
TARGETS = [
        "euler_chi",
    ]

# Conjecture on the target invariants using the stages defined above.
result = g3.conjecture(
    targets=TARGETS,
    stages=STAGES,
    include_invariant_products=False,
    include_abs=False,
    include_min_max=False,
    include_log=False,
    enable_sophie=True,
    sophie_stages=STAGES,
    quick=True,
    show=True,
)

Stage breakdown: {'euler_chi': {'constant': {'conjectures': 2, 'sophie': 4}, 'ratio': {'conjectures': 1, 'sophie': 1}, 'lp1': {'conjectures': 1, 'sophie': 1}, 'geom_mean': {'conjectures': 4, 'sophie': 5}}}
Total conjectures: 7
Total Sophie conditions: 9

=== Top conjectures (by touch_count, then support) ===

Conjecture 1. (w_gcd) ⇒ euler_chi ≤ 0   [touches=408, support=7555]

Conjecture 2. ((w_gcd) ∧ (has_repeat_weight)) ⇒ euler_chi ≤ ((4 · h11) + (-4 · √((h11 · h21))))   [touches=52, support=779]

Conjecture 3. ((w_gcd) ∧ (sum_divisible_by_all_weights)) ⇒ euler_chi ≤ 0   [touches=15, support=147]

Conjecture 4. ((w_gcd) ∧ (sum_divisible_by_all_weights)) ⇒ euler_chi ≤ ((4 · h11) + (-4 · √((h11 · h21))))   [touches=15, support=147]

Conjecture 5. ((w_gcd) ∧ (sum_divisible_by_all_weights)) ⇒ euler_chi ≤ ((-4 · w_sum) + (4 · √((w_sum · w_lcm))))   [touches=15, support=147]

Conjecture 6. (w_gcd) ⇒ euler_chi ≤ 960   [touches=4, support=7555]

Conjecture 7. ((w_gcd) ∧ (exactly_one_even_wei